In [1]:
import os
import pandas as pd
import nltk
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from nltk.classify import NaiveBayesClassifier
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

In [2]:
df = pd.read_csv(os.path.join('..', 'storage', 'base_sentiment.1.0.2.csv'))

In [3]:
df

,text,sentiment
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",negative
1,Este é um exemplo do motivo pelo qual a maiori...,negative
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",negative
3,Nem mesmo os Beatles puderam escrever músicas ...,negative
4,Filmes de fotos de latão não é uma palavra apr...,negative
...,...,...
49454,"Como a média de votos era muito baixa, e o fat...",positive
49455,O enredo teve algumas reviravoltas infelizes e...,positive
49456,Estou espantado com a forma como este filme e ...,positive
49457,A Christmas Together realmente veio antes do m...,positive


In [4]:
pos = list(df.query('sentiment == "positive"')['text'][:1000].values)
neg = list(df.query('sentiment == "negative"')['text'][:1000].values)

In [5]:
pos_docs = [(sentence, 'positive') for sentence in pos]
neg_docs = [(sentence, 'negative') for sentence in neg]

In [6]:
pos_docs[0]

('Eu fui e vi este filme ontem à noite depois de ser persuadido por alguns amigos meus. Eu admitiria que estava relutante em vê-lo porque, pelo que eu sabia de Ashton Kutcher, ele só conseguia fazer comédia. Eu estava errado. Kutcher interpretou o personagem de Jake Fischer muito bem, e Kevin Costner interpretou Ben Randall com tal profissionalismo. O sinal de um bom filme é que ele pode brincar com nossas emoções. Este fez exatamente isso. Todo o teatro que foi vendido foi superado pelo riso durante a primeira metade do filme, e foi levado às lágrimas durante o segundo semestre. Ao sair do teatro, eu não só vi muitas mulheres em lágrimas, mas também muitos homens adultos, tentando desesperadamente não deixar ninguém vê-los chorando. Este filme foi ótimo, e eu sugiro que você vá vê-lo antes de julgar.',
 'positive')

In [7]:
train_subj = pos_docs[:800]
test_subj = pos_docs[800:1000]
train_obj = neg_docs[:80]
test_obj = neg_docs[80:100]

training_docs = train_subj + train_obj
testing_docs = test_subj + test_obj

In [8]:
sent_analyzer = SentimentAnalyzer()
all_words_neg = sent_analyzer.all_words([mark_negation(doc) for doc in training_docs])

In [9]:
unigram_feats = sent_analyzer.unigram_word_feats(all_words_neg, min_freq=4)
len(unigram_feats)

89

In [10]:
sent_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)

In [11]:
training_set  = sent_analyzer.apply_features(training_docs)
testing_set = sent_analyzer.apply_features(testing_docs)

In [12]:
trainer = NaiveBayesClassifier.train
classifier = sent_analyzer.train(trainer, training_set)
for key, value in sorted(sent_analyzer.evaluate(testing_set).items()):
    print(f'{key}: {value}')

Training classifier
Evaluating NaiveBayesClassifier results...
Accuracy: 0.675
F-measure [negative]: 0.7346938775510203
F-measure [positive]: 0.5806451612903225
Precision [negative]: 0.6206896551724138
Precision [positive]: 0.8181818181818182
Recall [negative]: 0.9
Recall [positive]: 0.45


In [28]:
sentences = ["Bom dia!", "Péssimo dia", "como vai?", "eu estive bem mau!", "mal"]
# paragraph = "Nada, estive muito mau nessa semana!"
# tricky_sentences = ["eu estive bem mau!", "tudo de bom"]

In [24]:
# lines_list = tokenize.sent_tokenize(paragraph)
# sentences.extend(lines_list)
# sentences.extend(tricky_sentences)

In [29]:
for sentence in sentences:
    
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(sentence)
    
    print(sentence)
    
    for k in sorted(ss):
        print(f'{k}: {ss[k]}', end=' ')
    print()

Bom dia!
compound: 0.0 neg: 0.0 neu: 1.0 pos: 0.0 
Péssimo dia
compound: 0.0 neg: 0.0 neu: 1.0 pos: 0.0 
como vai?
compound: 0.0 neg: 0.0 neu: 1.0 pos: 0.0 
eu estive bem mau!
compound: 0.0 neg: 0.0 neu: 1.0 pos: 0.0 
mal
compound: 0.0 neg: 0.0 neu: 1.0 pos: 0.0 
